<a href="https://colab.research.google.com/github/DeveloperVivek9/Calorie-Tracker/blob/main/Deployment/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.9.0
!pip install torchvision==0.10.0
!pip install detecto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.


In [ ]:
import cv2
from detecto import core, utils
from detecto.core import Dataset,DataLoader,Model
from detecto.utils import read_image
from detecto.utils import xml_to_csv
from google.colab import drive
import numpy as np
import pandas as pd
import os
import shutil
import torch
from torchvision import transforms

In [ ]:
"""
drive.mount("/content/drive")
"""

Mounted at /content/drive


In [ ]:
def Initial_Training(base_path):
  """
  base_path to contain -> train_labels.csv, test_labels.csv, Annotated folder which is the image folder.
  We save the model in base_path.
  """
  TRAIN_PATH = os.path.abspath(os.path.join(base_path,'Annotated'))
  TRAIN_LABEL_PATH = os.path.abspath(os.path.join(base_path,'train_labels.csv'))
  TEST_PATH = os.path.abspath(os.path.join(base_path,'Annotated'))
  TEST_LABEL_PATH = os.path.abspath(os.path.join(base_path,'test_labels.csv'))
  Train_dataset = Dataset(TRAIN_LABEL_PATH,TRAIN_PATH)
  Test_dataset = Dataset(TEST_LABEL_PATH,TEST_PATH)
  classes_list_df=pd.read_csv(base_path+'Classes_List.csv')
  classes_list=list(classes_list_df['Class Name'])
  torch.cuda.empty_cache()
  loader=DataLoader(Train_dataset, batch_size=1, shuffle=True)
  model = Model(classes_list)
  losses = model.fit(loader, Test_dataset, epochs=15, lr_step_size=5, learning_rate=0.001, verbose=True)
  model.save(base_path+'model_weights 0.pth')  

In [ ]:
def Online_Training(base_path,count):
  """
  base_path to contain -> train_labels.csv, test_labels.csv, 'Annotated' folder which contains the images.
  We save the model in base_path.
  """
  TRAIN_PATH = os.path.abspath(os.path.join(base_path,'Annotated'))
  TRAIN_LABEL_PATH = os.path.abspath(os.path.join(base_path,'train_labels.csv'))
  TEST_PATH = os.path.abspath(os.path.join(base_path,'Annotated'))
  TEST_LABEL_PATH = os.path.abspath(os.path.join(base_path,'test_labels.csv'))
  Train_dataset = Dataset(TRAIN_LABEL_PATH,TRAIN_PATH)
  Test_dataset = Dataset(TEST_LABEL_PATH,TEST_PATH)
  classes_list_df=pd.read_csv(base_path+'Classes_List.csv')
  classes_list=list(classes_list_df['Class Name'])
  torch.cuda.empty_cache()
  loader=DataLoader(Train_dataset, batch_size=1, shuffle=True)
  model = Model.load(base_path+'model_weights '+str(count-1)+'.pth', classes_list)
  losses = model.fit(loader, Test_dataset, epochs=10, lr_step_size=5, learning_rate=0.00001, verbose=True)
  model.save(base_path+'model_weights '+str(count)+'.pth')  

In [ ]:
"""
base_path="/content/drive/MyDrive/Experiment/"
Initial_Training(base_path)
"""

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

It looks like you're training your model on a CPU. Consider switching to a GPU; otherwise, this method can take hours upon hours or even days to finish. For more information, see https://detecto.readthedocs.io/en/latest/usage/quickstart.html#technical-requirements
Epoch 1 of 1
Begin iterating over training dataset


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 3/3 [01:03<00:00, 21.21s/it]


Begin iterating over validation dataset


100%|██████████| 1/1 [00:08<00:00,  8.04s/it]


Loss: 0.9722896814346313
